In [306]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown
from ordered_set import OrderedSet

In [307]:
base_df = pd.read_csv('base_df.csv')
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')

In [308]:
def return_multiple_author(base_df):
    testing_df = base_df[base_df['author'].notnull()]
    author = testing_df['author'].tolist()
    author1 = [i.replace('[Provider:earticle]','').strip() for i in author]
    testing_df['journal'] = author1
    return testing_df.loc[:,['author','journal','paper','year','from']]

In [309]:
fm_base_df = return_mutiple_author(fm_base_df)
f_base_df = return_mutiple_author(f_base_df)
sc_base_df = return_mutiple_author(sc_base_df)
dr_base_df = return_mutiple_author(dr_base_df)

In [310]:
fm_base_df['year'] = fm_base_df['year'].apply(lambda x : str(x).split('.')[0])
f_base_df['year'] = f_base_df['year'].apply(lambda x : str(x).split('.')[0])
sc_base_df['year'] = sc_base_df['year'].apply(lambda x : str(x).split('.')[0])
dr_base_df['year'] = dr_base_df['year'].apply(lambda x : str(x).split('.')[0])

In [311]:
counter_df = pd.DataFrame.from_dict(Counter(fm_base_df['author'].tolist() + \
f_base_df['author'].tolist() + \
sc_base_df['author'].tolist() + \
dr_base_df['author'].tolist()),orient='index',columns=['count']).sort_values('count',ascending=False)

counter_df.head(10)

,count
Fama E. F. and K. R. French,192
Jensen M. and W. Meckling,75
Carhart Mark M.,74
Black F.,74
Fama E. F. and J. D. MacBeth,70
고봉찬 김진우,68
Fama E. and French K.,64
Fama Eugene F. Kenneth R. French,59
Jegadeesh N. and S. Titman,53
Ang A. R. J. Hodrick Y. Xing and X. Zhang,50


In [312]:
catch_author_ls = counter_df[counter_df['count'] > 3].index.tolist()
print(len(catch_author_ls))
catch_author_ls[:5]

1237


['Fama E. F. and K. R. French ',
 'Jensen M. and W. Meckling ',
 'Carhart Mark M. ',
 'Black F. ',
 'Fama E. F. and J. D. MacBeth ']

In [313]:
catch_author_ls_copy = catch_author_ls.copy()
tuned_catch_author_ls = []
excepted_ls = []
similar_match_ls = []

for i in range(5) : 
    print(i)
    for idx1,val1 in enumerate(catch_author_ls_copy) : 
        for idx2,val2 in enumerate(catch_author_ls_copy) :
            if idx1 != idx2 : 
                v1 = set(val1.split(' '))
                v2 = set(val2.split(' '))
                try : 
                    if len(set(v1).intersection(v2)) / ((len(v1) + len(v2))/2) > 0.85 : 
                        similar_match_ls.append((val1,val2))
                        if len(v1) > len(v2) :
                            catch_author_ls_copy.pop(catch_author_ls_copy.index(val2))
                            excepted_ls.append((val2))
                        else :
                            catch_author_ls_copy.pop(catch_author_ls_copy.index(val1))
                            excepted_ls.append(((val1)))                    
                except : pass

len(catch_author_ls) , len(catch_author_ls_copy) , len(excepted_ls)

0
1
2
3
4


(1237, 1064, 173)

In [314]:
[i for i in catch_author_ls_copy if 'fama' in i.lower()]

['Fama E. F. and K. R. French ',
 '[Provider:earticle]  Fama E. F. and J. D. MacBeth ',
 'Fama E. F. ',
 'Fama Eugene F. and French Kenneth R. ',
 'Fama E. F. and M. C. Jensen ']

In [315]:
ready_to_target_dict = dict(zip(catch_author_ls_copy,[[] for i in range(len(catch_author_ls_copy))]))
append = 0
naive_append = 0

for idx1,val1 in enumerate(base_df['author'].tolist()) : 
    if idx1 % 4000 == 0 : print(idx1,end='\r')
    for idx2,val2 in enumerate(ready_to_target_dict.keys()) :
        
        if type(val1) == str:
            paper_name = val1.lower()
            target_name = val2.lower()

            paper_name_to_word = set(paper_name.split(' '))
            target_name_to_word = set(target_name.split(' '))
            if len(paper_name_to_word.intersection(target_name_to_word)) / \
            ((len(paper_name_to_word) + len(target_name_to_word))/2) > 0.85 : 
                ready_to_target_dict[val2].append(val1)
                append += 1

In [316]:
final_ls = []
process_ls = []
error_ls = []

for idx1 , val in enumerate(base_df['author']) : 
    process_ls = []
    for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
        if val in ready_to_target_dict.get(key_) : 
            process_ls.append(key_)
    if type(val) == str :
        if len(process_ls) == 1 :
            final_ls.append(process_ls[0])
        elif len(process_ls) == 0 :
            final_ls.append(val)
        else : final_ls.append(val)
    else : final_ls.append(None)
        
len(final_ls) == len(base_df)

True

In [317]:
base_df['new_author'] = final_ls

In [318]:
len(set(final_ls)) , len(set(base_df['author']))

(9983, 10536)

In [319]:
final_author_df = base_df.copy()
final_author_df = final_author_df[final_author_df['new_author'].notnull()]

In [320]:
display(Markdown('### 기존의 방법론'))
display(pd.DataFrame.from_dict(Counter(final_author_df['author']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))
display(Markdown('### 개선된 방법론'))
display(pd.DataFrame.from_dict(Counter(final_author_df['new_author']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))

### 기존의 방법론

,count
Fama E. F. and K. R. French,169
Jensen M. and W. Meckling,63
Black F.,58
Carhart Mark M.,50
Fama E. F. and J. D. MacBeth,49
Ang A. R. J. Hodrick Y. Xing and X. Zhang,43
Black F. and M. Scholes,41
Fama E. and French K.,41
Bollerslev T.,38
Jegadeesh N. and S. Titman,38


### 개선된 방법론

,count
Fama E. F. and K. R. French,242
Jensen M. C. and W. J. Meckling,109
Carhart Mark M.,75
[Provider:earticle] Fama E. F. and J. D. MacBeth,67
Jensen M. C.,60
Black F.,58
Black F. and Scholes M.,58
Bakshi G. N. Kapadia and D. Madan,54
Barber B. M. T. Odean and N. Zhu,52
Choe H. B. C. Kho and R. M. Stulz,47


In [321]:
def return_df_author_frequency_table_under_journal(base_df):
    final_ls = []
    process_ls = []

    for idx1 , val in enumerate(base_df['author']) : 
        process_ls = []
        for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
            if val in ready_to_target_dict.get(key_) : 
                process_ls.append(key_)
        if type(val) == str :
            if len(process_ls) == 1 :
                final_ls.append(process_ls[0])
            elif len(process_ls) == 0 :
                final_ls.append(val)
            else : final_ls.append(val)
        else : final_ls.append(None)

    if len(final_ls) == len(base_df) : 
        base_df['new_author'] = final_ls
    else : print('error')
    
    print("lower uniqueness means better performance, orgin uniqueness : {} , revised uniqueness : {}"\
          .format(len(set(base_df['author'])),len(set(final_ls))))
    return base_df

In [322]:
fm_final_df = return_df_author_frequency_table_under_journal(fm_base_df)
f_final_df = return_df_author_frequency_table_under_journal(f_base_df)
sc_final_df = return_df_author_frequency_table_under_journal(sc_base_df)
dr_final_df = return_df_author_frequency_table_under_journal(dr_base_df)

lower uniqueness means better performance, orgin uniqueness : 6313 , revised uniqueness : 5998
lower uniqueness means better performance, orgin uniqueness : 3603 , revised uniqueness : 3393
lower uniqueness means better performance, orgin uniqueness : 2615 , revised uniqueness : 2392
lower uniqueness means better performance, orgin uniqueness : 1536 , revised uniqueness : 1467


In [323]:
def return_new_counter_df(base_df):
    return pd.DataFrame.from_dict(Counter(base_df['new_author'].tolist()),orient='index',columns=['count']).sort_values('count',ascending=False)

In [324]:
display(return_new_counter_df(fm_final_df).head(10))
display(return_new_counter_df(f_final_df).head(10))
display(return_new_counter_df(sc_final_df).head(10))
display(return_new_counter_df(dr_final_df).head(10))

,count
Fama E. F. and K. R. French,89
Jensen M. C.,41
Jensen M. C. and W. J. Meckling,36
Newey W. and West K. D.,27
Myers S. C. and N. S. Majluf,25
Myers S. C.,24
Fama E. F.,22
Opler T. Pinkowitz L. Stulz R. and Williamson R.,21
설원식 김수정,21
Black F.,20


,count
Fama E. F. and K. R. French,37
Fama Eugene F. and French Kenneth R.,22
Jensen M. C.,20
[Provider:earticle] Fama E. F. and J. D. MacBeth,19
Merton R. C.,17
Keynes J. M.,17
Myers S. C. and N. S. Majluf,15
Jegadeesh N. and S. Titman,15
Kyle A. S.,14
Carhart Mark M.,13


,count
Fama E. F. and K. R. French,136
Jensen M. C. and W. J. Meckling,86
Carhart Mark M.,67
[Provider:earticle] Fama E. F. and J. D. MacBeth,51
Black F. and Scholes M.,48
Barber B. M. T. Odean and N. Zhu,35
고봉찬 김진우,33
Choe H. B. C. Kho and R. M. Stulz,33
Barber B. M. and J. D. Lyon,32
Fama Eugene F. and French Kenneth R.,31


,count
Fama E. F. and K. R. French,41
Bakshi G. N. Kapadia and D. Madan,33
Sutcliffe C.,25
Black F.,24
Cifuentes A. and G. Katsaros 2007 The One-Factor Gaussian Copula Applied to CDOs: Just Say NO (or if you see a correlation smile she is laughing at your,16
Cochrane J. H.,16
이우백,14
이재하 한덕희,14
477 대기업집단 내부거래의 대리인가설 검정에 관한 연구 Agency Costs of Self-Dealings in Korean Business Groups* ** Won Heum Lee Hongik University Abstract25) I examine the agency problems of self-dealings in the Korean business groups (hereafter referred to as,13
Newey W. and West K. D.,13
